In [1]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px
from utils.settings import DETERMINISTIC_MODELS, DETERMINISTIC_VARS, ENSEMBLE_VARS, ENSEMBLE_MODELS

In [2]:
def get_ensemble_data(latitude=53.55,
                      longitude=9.99,
                      variables=",".join(ENSEMBLE_VARS),
                      timezone='auto',
                      model=ENSEMBLE_MODELS[0]['value'],
                      from_now=True):
    """
    Get the ensemble data
    """
    if model == 'icon_seamless':
        forecast_days = 7
    else:
        forecast_days = 14
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    resp = r.get("https://ensemble-api.open-meteo.com/v1/ensemble",
                 params=payload)
    resp.raise_for_status()
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'], ambiguous='NaT', nonexistent='NaT')

    data = data.dropna(subset=data.columns[data.columns != 'time'],
                       how='all')
    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime(
            'now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

In [7]:
df = get_ensemble_data(latitude=44, longitude=53, model='icon_eu', variables='wind_direction_10m')

In [200]:
def make_barpolar_figure(df, n_partitions=15, bins=np.linspace(0, 360, 15)):
    timeSpan = (df.time.iloc[-1]-df.time.iloc[0])
    rule = int((timeSpan.total_seconds()/3600.)/n_partitions)
    subset = df.resample(str(rule)+"H", on='time').first()
    subset = subset.loc[:, subset.columns.str.contains('wind_direction')]

    out = []
    for i, row in subset.iterrows():
        out.append(pd.cut(row.values, bins=bins).value_counts().values)
    # Convert to normalized percentage
    for i, o in enumerate(out):
        out[i] = (o / len(subset.columns)) * 100.    
    n_plots = len(out) - 1
    fig = make_subplots(rows=1,
                        cols=n_plots,
                        specs=[[{'type': 'polar'} for _ in range(n_plots)]],
                        horizontal_spacing=0)
    for i in range(n_plots):
        fig.add_trace(go.Barpolar(
            r=out[i-1],
            theta=bins,
            marker_color='rgb(106,81,163)',
            showlegend=False), row=1, col=i+1)
    fig.update_polars(radialaxis_showticklabels=False,
                    angularaxis_showticklabels=False)
    fig.update_layout(margin={"r": 2, "t": 1, "l": 2, "b": 0.1},
                    height=100,
                    )
    #fig.show(config=dict(displayModeBar=False))
    
    return fig

In [201]:
make_barpolar_figure(df)

In [204]:
images_config.update(dict(displayModeBar=False))

In [207]:
{**images_config, 'displayModeBar': False}

{'toImageButtonOptions': {'format': 'png',
  'height': None,
  'width': None,
  'scale': 1.5},
 'displayModeBar': False}